# Gradient Based Constraint Learning Demo

Licensed under the Apache License, Version 2.0.

This colab explores joint learning neural networks with soft constraints.

In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf

from tensorflow import keras

2021-11-12 11:54:28.931214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 11:54:28.931232: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Dataset and Task

We test and validate our system over a common fairness dataset and task: [Adult Census Income dataset](https://archive.ics.uci.edu/ml/datasets/Census+Income). This data was extracted from the [1994 Census bureau database](http://www.census.gov/en.html) by Ronny Kohavi and Barry Becker. Our analysis aims at learning a model that does not bias predictions towards men over 50K through soft constraints.

In [2]:
# ========================================================================
# Constants
# ========================================================================
_TRAIN_PATH = ''
_TEST_PATH = ''

_COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "gender",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "income_bracket"]

# ========================================================================
# Seed Data
# ========================================================================
SEED = random.randint(-10000000, 10000000)
print("Seed: %d" % SEED)
tf.random.set_seed(SEED)

# ========================================================================
# Load Data
# ========================================================================
with tf.io.gfile.GFile(_TRAIN_PATH, 'r') as csv_file:
  train_df = pd.read_csv(csv_file, names=_COLUMNS, sep=r'\s*,\s*', na_values="?").dropna(how="any", axis=0)

with tf.io.gfile.GFile(_TEST_PATH, 'r') as csv_file:
  test_df = pd.read_csv(csv_file, names=_COLUMNS, skiprows=[0], sep=r'\s*,\s*', na_values="?").dropna(how="any", axis=0)

Seed: 4649844


/tmp/ipykernel_20720/1001939022.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv(csv_file, names=_COLUMNS, sep=r'\s*,\s*', na_values="?").dropna(how="any", axis=0)


NotFoundError: ; No such file or directory

# Feature Columns

The following code was taken from [intro_to_fairness](https://colab.sandbox.google.com/notebooks/mlcc/intro_to_fairness.ipynb#scrollTo=tAG5hUJwx725). In short, Tensorflow requires a mapping of data and so every column is specified.

In [3]:
#@title Prepare Dataset
# ========================================================================
# Categorical Feature Columns
# ========================================================================
# Unknown length
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    "occupation", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket(
    "native_country", hash_bucket_size=1000)

# Known length
gender = tf.feature_column.categorical_column_with_vocabulary_list(
    "gender", ["Female", "Male"])
race = tf.feature_column.categorical_column_with_vocabulary_list(
    "race", [
        "White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"
    ])
education = tf.feature_column.categorical_column_with_vocabulary_list(
    "education", [
        "Bachelors", "HS-grad", "11th", "Masters", "9th",
        "Some-college", "Assoc-acdm", "Assoc-voc", "7th-8th",
        "Doctorate", "Prof-school", "5th-6th", "10th", "1st-4th",
        "Preschool", "12th"
    ])
marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    "marital_status", [
        "Married-civ-spouse", "Divorced", "Married-spouse-absent",
        "Never-married", "Separated", "Married-AF-spouse", "Widowed"
    ])
relationship = tf.feature_column.categorical_column_with_vocabulary_list(
    "relationship", [
        "Husband", "Not-in-family", "Wife", "Own-child", "Unmarried",
        "Other-relative"
    ])
workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    "workclass", [
        "Self-emp-not-inc", "Private", "State-gov", "Federal-gov",
        "Local-gov", "?", "Self-emp-inc", "Without-pay", "Never-worked"
    ])

# ========================================================================
# Numeric Feature Columns
# ========================================================================
age = tf.feature_column.numeric_column("age")
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
fnlwgt = tf.feature_column.numeric_column("fnlwgt")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

# ========================================================================
# Specify Features
# ========================================================================
deep_columns = [
    tf.feature_column.indicator_column(workclass),
    tf.feature_column.indicator_column(education),
    tf.feature_column.indicator_column(age_buckets),
    tf.feature_column.indicator_column(gender),
    tf.feature_column.indicator_column(relationship),
    tf.feature_column.embedding_column(native_country, dimension=8),
    tf.feature_column.embedding_column(occupation, dimension=8),
]

features = {
  'age': tf.keras.Input(shape=(1,), name='age'),
  'education': tf.keras.Input(shape=(1,), name='education', dtype=tf.string),
  'gender': tf.keras.Input(shape=(1,), name='gender', dtype=tf.string),
  'native_country': tf.keras.Input(shape=(1,), name='native_country', dtype=tf.string),
  'occupation': tf.keras.Input(shape=(1,), name='occupation', dtype=tf.string),
  'relationship': tf.keras.Input(shape=(1,), name='relationship', dtype=tf.string),
  'workclass': tf.keras.Input(shape=(1,), name='workclass', dtype=tf.string),
}

# ========================================================================
# Create Dataset
# ========================================================================
def df_to_dataset(dataframe, shuffle=True, batch_size=512):
    dataframe = dataframe.copy()
    labels = dataframe.pop('income_bracket').apply(lambda x: ">50K" in x).astype(int)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [4]:
#@title Helper Functions
def confusion_matrix(predictions, labels, threshold=0.5):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for prediction, label in zip(predictions, labels):
        if prediction > threshold:
            if label == 1:
                tp += 1
            else:
                fp += 1
    else:
        if label == 0:
            tn += 1
        else:
            fn += 1
    return (tp, tn, fp, fn)

def remove_group(dataframe, predictions, group):
    dataframe = dataframe.copy()
    dataframe['predictions'] = predictions
    dataframe = dataframe[dataframe.gender != group]

    group_predictions = dataframe.pop('predictions')

    return dataframe, group_predictions

def print_accuracy(dataframe, predictions, threshold=0.5):
    dataframe = dataframe.copy()
    labels = dataframe.pop('income_bracket').apply(lambda x: ">50K" in x).astype(int)

    tp, tn, fp, fn = confusion_matrix(predictions, labels, threshold=threshold)
    print("True Positives: %d True Negatives: %d False Positives %d False Negatives: %d" % (tp, tn, fp, fn))
    print("Accuracy: %0.5f" % ((tp+tn) / (tp + tn + fp + fn)))
    print("Positive Accuracy: %0.5f" % (tp / (tp + fp)))
    print("Negative Accuracy: %0.5f" % (tn / (tn + fn)))
    print("Percentage Predicted over >50K: %0.5f" % (((tp + fp) / (tp + tn + fp + fn)) * 100))

    return (tp, tn, fp, fn)

def parity(m_tp, m_fp, m_tn, m_fn, f_tp, f_fp, f_tn, f_fn):
    return ((m_tp + m_fp) / (m_tp + m_tn + m_fp + m_fn)) - ((f_tp + f_fp) / (f_tp + f_tn + f_fp + f_fn))

def print_title(title, print_length=50):
    print(('-' * print_length) + '\n' + title + '\n' + ('-' * print_length))

def print_analysis(train_df, train_predictions, test_df, test_predictions):
    print_title("Train Accuracy")
    print_accuracy(train_df, train_predictions)

    print_title("Full Test Accuracy")
    print_accuracy(test_df, test_predictions)

    print_title("Male Test Accuracy")
    male_df, male_pred = remove_group(test_df, test_predictions, "Female")
    m_tp, m_tn, m_fp, m_fn = print_accuracy(male_df, male_pred)

    print_title("Female Test Accuracy")
    female_df, female_pred = remove_group(test_df, test_predictions, "Male")
    f_tp, f_tn, f_fp, f_fn = print_accuracy(female_df, female_pred)

    print_title("Parity")
    print(parity(m_tp, m_fp, m_tn, m_fn, f_tp, f_fp, f_tn, f_fn))

# Create and Run Non-Constrained Neural Model

Defining our neural model that will be used as a comparison. Note: this model was purposfully designed to be simplistic, as it is trying to highlight the benifit to learning with soft constraints.

In [5]:
def build_model(feature_columns, features):
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
    hidden_layer_1 = tf.keras.layers.Dense(1024, activation='relu')(feature_layer(features))
    hidden_layer_2 = tf.keras.layers.Dense(512, activation='relu')(hidden_layer_1)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer_2)

    model = tf.keras.Model([v for v in features.values()], output)

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])

    return model

baseline_model = build_model(deep_columns, features)
baseline_model.fit(df_to_dataset(train_df), epochs=50)

test_predictions = baseline_model.predict(df_to_dataset(test_df, shuffle=False))
baseline_model.evaluate(df_to_dataset(test_df))

train_predictions = baseline_model.predict(df_to_dataset(train_df, shuffle=False))
baseline_model.evaluate(df_to_dataset(train_df))

2021-11-12 11:55:49.884441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 11:55:49.885220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 11:55:49.885270: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-12 11:55:49.885313: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-12 11:55:49.885357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

NameError: name 'train_df' is not defined

# Analyze Non-Constrained Results

For this example we look at the fairness constraint that the protected group (gender) should have no predictive difference between classes. In this situation this means that the ratio of positive predictions should be the same between male and female.

Note: this is by no means the only fairness constraint needed to have a fair model, and in fact can result in some doubious results (as seen in the follwoing section).

The results do clearly show a skew in ratios as males are have a higher ratio of >50k predictions.

In [ ]:
print_analysis(train_df, train_predictions, test_df, test_predictions)

# Define Constraints

This requires a constrained loss function and a custom train step within the keras model class.

In [ ]:
def constrained_loss(data, logits, threshold=0.5, weight=3):
    """Linear constrained loss for equal ratio prediction for the protected group.

    The constraint: (#Female >50k / #Total Female) - (#Male >50k / #Total Male)
    This constraint penalizes predictions between the protected group (gender),
    such that the ratio between all classes must be the same.
    An important note: to maintian differentability we do not use #Female >50k
    (which requires a round operation), instead we set values below the threshold
    to zero, and sum the logits.

    Args:
    data: Input features.
    logits: Predictions made in the logit.
    threshold: Binary threshold for predicting positive and negative labels.
    weight: Weight of the constrained loss.

    Returns:
    A scalar loss of the constraint violations.
    """
    gender_label, gender_idx, gender_count = tf.unique_with_counts(data['gender'], out_idx=tf.int32, name=None)
    cut_logits = tf.reshape(tf.cast(logits > threshold, logits.dtype) * logits, [-1])

    def f1():
        return gender_idx
    def f2():
        return tf.cast(tf.math.logical_not(tf.cast(gender_idx, tf.bool)), tf.int32)

    # Load male indexes as ones and female indexes to zeros.
    male_index = tf.cond(tf.reduce_all(tf.equal(gender_label, tf.constant(["Male", "Female"]))), f1, f2)
    # Cast the integers to float32 to do a multiplication with the logits.
    male_index = tf.cast(male_index, tf.float32)
    # (#Male > 50k / #Total Male)
    male_prob = tf.divide(tf.reduce_sum(tf.multiply(cut_logits, male_index)), tf.reduce_sum(male_index))

    # Flip all female indexes to one and male indexes to zeros.
    female_index = tf.math.logical_not(tf.cast(male_index, tf.bool))
    # Cast the integers to float32 to do a multiplication with the logits.
    female_index = tf.cast(female_index, tf.float32)
    # (#Female > 50k / #Total Female)
    female_prob = tf.divide(tf.reduce_sum(tf.multiply(cut_logits, female_index)), tf.reduce_sum(female_index))

    # Since tf.math.abs is not differentable, separate the loss into two hinges.
    loss = tf.add(tf.maximum(male_prob - female_prob, 0.0), tf.maximum(female_prob - male_prob, 0.0))
    return tf.multiply(loss, weight)

class StructureModel(keras.Model):
    def train_step(self, data):
        features, labels = data

        with tf.GradientTape() as tape:
            logits = self(features, training=True)
            standard_loss = self.compiled_loss(labels, logits, regularization_losses=self.losses)
            constraint_loss = constrained_loss(features, logits)
            loss =  standard_loss + constraint_loss

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(labels, logits)

        return {m.name: m.result() for m in self.metrics}

# Build and Run Constrained Neural Model

In [ ]:
def build_constrained_model(feature_columns, features):
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
    hidden_layer_1 = tf.keras.layers.Dense(1024, activation='relu')(feature_layer(features))
    hidden_layer_2 = tf.keras.layers.Dense(512, activation='relu')(hidden_layer_1)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer_2)

    model = StructureModel([v for v in features.values()], output)

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])

    return model

constrained_model = build_constrained_model(deep_columns, features)
constrained_model.fit(df_to_dataset(train_df), epochs=50)

test_predictions = constrained_model.predict(df_to_dataset(test_df, shuffle=False))
constrained_model.evaluate(df_to_dataset(test_df))

train_predictions = constrained_model.predict(df_to_dataset(train_df, shuffle=False))
constrained_model.evaluate(df_to_dataset(train_df))

# Analyze Constrained Results

Ideally this constraint should correct the ratio imbalance between the protected groups (gender). This means our parity should be very close to zero.

Note: This constraint does not mean the neural classifier is guaranteed to generalize and make better predictions. It is more likely to attempt to balance the class prediction ratio in the simplest fashion (resulting in a worse accuracy).

In [ ]:
print_analysis(train_df, train_predictions, test_df, test_predictions)